##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar o treinamento de múltiplos workers com CPUs/GPUs

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este guia demonstra como migrar seu workflow de treinamento distribuído de múltiplos workers do TensorFlow 1 para o TensorFlow 2.

Para realizar treinamento multi-worker com CPUs/GPUs:

- No TensorFlow 1, você geralmente usa as APIs `tf.estimator.train_and_evaluate` e `tf.estimator.Estimator`.
- No TensorFlow 2, use as APIs Keras para escrever o modelo, a função de perda, o otimizador e as métricas. Em seguida, distribua o treinamento com a API Keras `Model.fit` ou com um loop de treinamento personalizado (com `tf.GradientTape`) entre múltiplos workers com `tf.distribute.experimental.ParameterServerStrategy` ou `tf.distribute.MultiWorkerMirroredStrategy`. Para mais detalhes, consulte os seguintes tutoriais:
    - [Treinamento distribuído com TensorFlow](../../guide/distributed_training.ipynb)
    - [Treinamento do servidor de parâmetros com Keras Model.fit / um loop de treinamento personalizado](../../tutorials/distribute/parameter_server_training.ipynb)
    - [MultiWorkerMirroredStrategy com Keras Model.fit](../../tutorials/distribute/multi_worker_with_keras.ipynb)
    - [MultiWorkerMirroredStrategy com um loop de treinamento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb).

## Configuração

Comece com alguns imports necessários e um dataset simples para fins de demonstração:

In [ ]:
# The notebook uses a dataset instance for `Model.fit` with
# `ParameterServerStrategy`, which depends on symbols in TF 2.7.
# Install a utility needed for this demonstration
!pip install portpicker

import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

Você vai precisar da variável de ambiente de configuração `'TF_CONFIG'` para treinar em múltiplas máquinas no TensorFlow. Use `'TF_CONFIG'` para especificar os endereços `'cluster'` e `'task'`. (Saiba mais no guia [Treinamento distribuído](../...guide/distributed_training.ipynb).)

In [ ]:
import json
import os

tf_config = {
    'cluster': {
        'chief': ['localhost:11111'],
        'worker': ['localhost:12345', 'localhost:23456', 'localhost:21212'],
        'ps': ['localhost:12121', 'localhost:13131'],
    },
    'task': {'type': 'chief', 'index': 0}
}

os.environ['TF_CONFIG'] = json.dumps(tf_config)

Observação: infelizmente, já que o treinamento multi-worker com APIs `tf.estimator` no TensorFlow 1 requer múltiplos clientes (o que seria bastante complicado de ser feito aqui neste notebook Colab), você vai tornar o notebook executável sem uma variável de ambiente `'TF_CONFIG'`, então ele volta para o treinamento local. (Saiba mais na seção <em data-md-type="emphasis">Configurando a variável de ambiente `'TF_CONFIG'`</em> no guia [Treinamento distribuído com TensorFlow](../../guide/distributed_training.ipynb).)

Use a instrução `del` para remover a variável (mas num treinamento multi-worker real no TensorFlow 1, você não precisa fazer isso):

In [ ]:
del os.environ['TF_CONFIG']

## TensorFlow 1: treinamento distribuído multi-worker com APIs tf.estimator

O trecho de código a seguir demonstra o fluxo de trabalho canônico do treinamento multi-worker no TF1: você usará um `tf.estimator.Estimator`, um `tf.estimator.TrainSpec`, um `tf.estimator.EvalSpec` e a API `tf.estimator.train_and_evaluate` para distribuir o treinamento:

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: treinamento multi-worker com estratégias de distribuição

No TensorFlow 2, o treinamento distribuído por múltiplos workers com CPUs, GPUs e TPUs é feito via estratégias `tf.distribute.Strategy`.

O exemplo a seguir demonstra como usar duas dessas estratégias: `tf.distribute.experimental.ParameterServerStrategy` e `tf.distribute.MultiWorkerMirroredStrategy`, ambas projetadas para o treinamento de CPU/GPU com múltiplos workers.

O `ParameterServerStrategy` emprega um *coordenador* (`'chief'`), o deixa ele mais amigável com o ambiente deste notebook Colab. Você usará alguns utilitários aqui para configurar os elementos de suporte essenciais para uma experiência executável aqui: você criará um *cluster no mesmo processo*, onde threads são usados ​​para simular os servidores de parâmetro (`'ps'`) e workers (`'worker'`). Para mais informações sobre treinamento do servidor de parâmetros, consulte o tutorial [Treinamento do servidor de parâmetros com ParameterServerStrategy](../../tutorials/distribute/parameter_server_training.ipynb).

Neste exemplo, primeiro defina a variável de ambiente `'TF_CONFIG'` com um `tf.distribute.cluster_resolver.TFConfigClusterResolver` para fornecer as informações do cluster. Se você estiver usando um sistema de gerenciamento de cluster para seu treinamento distribuído, verifique se ele já fornece `'TF_CONFIG'` para você; nesse caso, você não precisa definir explicitamente essa variável de ambiente. (Saiba mais na seção <em data-md-type="emphasis">Configurando a variável de ambiente `'TF_CONFIG'`</em> no guia [Treinamento distribuído com TensorFlow](../../guide/distributed_training.ipynb).)

In [ ]:
# Find ports that are available for the `'chief'` (the coordinator),
# `'worker'`s, and `'ps'` (parameter servers).
import portpicker

chief_port = portpicker.pick_unused_port()
worker_ports = [portpicker.pick_unused_port() for _ in range(3)]
ps_ports = [portpicker.pick_unused_port() for _ in range(2)]

# Dump the cluster information to `'TF_CONFIG'`.
tf_config = {
    'cluster': {
        'chief': ["localhost:%s" % chief_port],
        'worker': ["localhost:%s" % port for port in worker_ports],
        'ps':  ["localhost:%s" % port for port in ps_ports],
    },
    'task': {'type': 'chief', 'index': 0}
}
os.environ['TF_CONFIG'] = json.dumps(tf_config)

# Use a cluster resolver to bridge the information to the strategy created below.
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

Em seguida, crie um `tf.distribute.Server` para cada worker e servidor de parâmetro um por um:

In [ ]:
# Workers need some inter_ops threads to work properly.
# This is only needed for this notebook to demo. Real servers
# should not need this.
worker_config = tf.compat.v1.ConfigProto()
worker_config.inter_op_parallelism_threads = 4

for i in range(3):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="worker",
      task_index=i,
      config=worker_config)

for i in range(2):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="ps",
      task_index=i)

Num treinamento distribuído real, em vez de iniciar todos os `tf.distribute.Server` no coordenador, você usará várias máquinas, e cada uma das designadas como `"worker"` e `"ps"` (servidores de parâmetro) irá executar um `tf.distribute.Server`. Consulte a seção *Clusters no mundo real* no tutorial [Treinamento do servidor de parâmetros](../../tutorials/distribute/parameter_server_training.ipynb) para mais detalhes.

Com tudo pronto, crie o objeto `ParameterServerStrategy`:

In [ ]:
strategy = tf.distribute.experimental.ParameterServerStrategy(cluster_resolver)

Depois de criar um objeto de estratégia, defina o modelo, o otimizador e outras variáveis ​​e chame o Keras `Model.compile` na API `Strategy.scope` para distribuir o treinamento. (Consulte os documentos da API `Strategy.scope` para obter mais informações.)

Se você preferir personalizar seu treinamento, por exemplo, definindo os passos para frente e para trás, consulte a seção *Treinamento com um loop de treinamento personalizado* no Tutorial [Treinamento do servidor de parâmetros](../../tutorials/distribute/parameter_server_training.ipynb) para mais detalhes.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
      (features, labels)).shuffle(10).repeat().batch(64)

eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).repeat().batch(1)

with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.legacy.Adagrad(learning_rate=0.05)
  model.compile(optimizer, "mse")

model.fit(dataset, epochs=5, steps_per_epoch=10)

In [ ]:
model.evaluate(eval_dataset, steps=10, return_dict=True)

> **Particionadores (`tf.distribute.experimental.partitioners`)**
>
> A `ParameterServerStrategy` no TensorFlow 2 oferece suporte ao particionamento variável e oferece os mesmos particionadores do TensorFlow 1, com nomes menos confusos:
>
> - `tf.compat.v1.variable_axis_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.MaxSizePartitioner`: um particionador que mantém os fragmentos abaixo de um tamanho máximo).
> - `tf.compat.v1.min_max_variable_partitioner` -&gt; `tf.distribute.experimental.partitioners.MinSizePartitioner`: um particionador que aloca um tamanho mínimo por fragmento.
> - `tf.compat.v1.fixed_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.FixedShardsPartitioner`: um particionador que aloca um número fixo de fragmentos.

Como alternativa, você pode usar um objeto `MultiWorkerMirroredStrategy`:

In [ ]:
# To clean up the `TF_CONFIG` used for `ParameterServerStrategy`.
del os.environ['TF_CONFIG']
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Você pode substituir a estratégia usada acima por um objeto `MultiWorkerMirroredStrategy` para realizar o treinamento com esta estratégia.

Assim como as APIs `tf.estimator`, como `MultiWorkerMirroredStrategy` é uma estratégia multicliente, não existe uma maneira fácil de executar treinamento distribuído neste notebook Colab. Portanto, substituir o código acima por essa estratégia acaba rodando as coisas localmente. Os tutoriais de treinamento multi-worker [com Keras Model.fit](../../tutorials/distribute/multi_worker_with_keras.ipynb) / [um loop de treinamento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb) demonstram como executar o treinamento multi-worker com a variável `'TF_CONFIG'` configurada, com dois workers num host local no Colab. Na prática, você criaria múltiplos workers em endereços/portas IP externos e usaria a variável `'TF_CONFIG'` para especificar a configuração do cluster para cada worker.

## Próximos passos

Para saber mais sobre o treinamento multi-worker distribuído com `tf.distribute.experimental.ParameterServerStrategy` e `tf.distribute.MultiWorkerMirroredStrategy` no TensorFlow 2, considere os seguintes recursos:

- Tutorial: [Treinamento do servidor de parâmetros com ParameterServerStrategy e Keras Model.fit / um loop de treinamento personalizado](../../tutorials/distribute/parameter_server_training.ipynb)
- Tutorial: [Treinamento multi-worker com MultiWorkerMirroredStrategy e Keras Model.fit](../../tutorials/distribute/multi_worker_with_keras.ipynb)
- Tutorial: [Treinamento multi-worker com MultiWorkerMirroredStrategy e um loop de treinamento personalizado](../../tutorials/distribute/multi_worker_with_ctl.ipynb)
- Guia: [Treinamento distribuído com o TensorFlow](../../guide/distributed_training.ipynb)
- Guia: [Otimize o desempenho da GPU do TensorFlow com o TensorFlow Profiler](../../guide/gpu_performance_analysis.ipynb)
- Guia: [Use uma GPU](../../guide/gpu.ipynb) (na seção Usando múltiplas GPUs)